In [25]:
import sqlite3
import pandas as pd
from bs4 import BeautifulSoup
import requests 

conn = sqlite3.connect('data/movies.db')
cur = conn.cursor()

In [26]:
imdb_title_basics = pd.DataFrame(cur.execute("SELECT tconst, primary_title, genres FROM imdb_title_basics").fetchall())
imdb_title_basics.columns = [i[0] for i in cur.description]
imdb_title_basics.head(3)

,tconst,primary_title,genres
0,tt0063540,Sunghursh,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,Drama


In [27]:
gross = []
#put dataframe that includes tconst and primary title as parameter
def imdb_gross(imdb_data):
    for x in imdb_data:
       # tconst = imdb_data["tconst"][x]
        url = f'https://www.imdb.com/title/tt0088247/'
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'lxml')
    
        if soup.find_all("span", class_="ipc-metadata-list-item__list-content-item")[3].text 
        gross_us_and_ca = soup.find_all("span", class_="ipc-metadata-list-item__list-content-item")[3].text
        gross_worldwide = soup.find_all("span", class_="ipc-metadata-list-item__list-content-item")[6].text
        
        #make list that includes movie title of row folled by gross domestic and worldwide of row's tconst
        gross.append([imdb_data["primary_title"][x], gross_us_and_ca, gross_worldwide])
        return gross
imdb_gross(imdb_title_basics)

SyntaxError: invalid syntax (<ipython-input-27-37a0e19d170e>, line 10)

##Cruella tests

In [41]:
url = 'https://www.imdb.com/title/tt3228774/'
response = requests.get(url)
cruel_soup = BeautifulSoup(response.text, 'lxml')

In [42]:
boxoffice_categories = cruel_soup.find_all("span", class_="ipc-metadata-list-item__label")
for item in boxoffice_categories:
    print(item.text)

Director
Director
Director
Genres
Countries of origin
Language
Gross US & Canada
Opening weekend US & Canada
Gross worldwide
Runtime
Color
Sound mix
Aspect ratio


In [43]:
gross_us_and_ca = cruel_soup.find_all("span", class_="ipc-metadata-list-item__list-content-item")
for item in gross_us_and_ca:
    print(item.text)

Hello cruel world.
Rated PG-13 for some violence and thematic elements
Cruella de Vil
$43,443,602
$21,496,997
May 30, 2021
$86,843,602
2h 14min
1.85 : 1


In [28]:
url = 'https://www.imdb.com/title/tt0088247/'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')

In [29]:
for items in soup.find_all("h3", class_="ipc-title__text"):
    print(items.text)
    if 

SyntaxError: invalid syntax (<ipython-input-29-986599d1ed82>, line 3)

In [34]:
soup.find_all("span", class_="ipc-metadata-list-item__label")

[<span class="ipc-metadata-list-item__label">Director</span>,
 <span class="ipc-metadata-list-item__label">Writers</span>,
 <span class="ipc-metadata-list-item__label">Director</span>,
 <span class="ipc-metadata-list-item__label">Writers</span>,
 <span class="ipc-metadata-list-item__label">Director</span>,
 <span class="ipc-metadata-list-item__label">Writers</span>,
 <span class="ipc-metadata-list-item__label">Genres</span>,
 <span class="ipc-metadata-list-item__label">Countries of origin</span>,
 <span class="ipc-metadata-list-item__label">Official site</span>,
 <span class="ipc-metadata-list-item__label">Languages</span>,
 <span class="ipc-metadata-list-item__label">Budget</span>,
 <span class="ipc-metadata-list-item__label">Gross US &amp; Canada</span>,
 <span class="ipc-metadata-list-item__label">Opening weekend US &amp; Canada</span>,
 <span class="ipc-metadata-list-item__label">Gross worldwide</span>,
 <span class="ipc-metadata-list-item__label">Runtime</span>,
 <span class="ipc-

In [39]:
boxoffice_categories = soup.find_all("span", class_="ipc-metadata-list-item__label")
for item in boxoffice_categories:
    print(item.text)

Director
Writers
Director
Writers
Director
Writers
Genres
Countries of origin
Official site
Languages
Budget
Gross US & Canada
Opening weekend US & Canada
Gross worldwide
Runtime
Color
Sound mix
Aspect ratio


In [30]:
gross_us_and_ca = soup.find_all("span", class_="ipc-metadata-list-item__list-content-item")
for item in gross_us_and_ca:
    print(item.text)
#gross_worldwide = soup.find_all("span", class_="ipc-metadata-list-item__list-content-item")[6].text
#print(gross_us_and_ca)

8 wins & 6 nominations
La sua missione e una sola: distruggere, uccidere... (His one and only mission: to destroy, to kill...) (Italian DVD)
R
Terminator
$6,400,000 (estimated)
$38,371,200
$4,020,663
Oct 28, 1984
$78,371,200
1h 47min
1.85 : 1
